Config:

In [1]:
import csv, re, datetime
from getpass import getpass                # Allows for an entry field to avoid hard-coded API keys
from cognite.client import CogniteClient   # Python SDK
from cognite.client.utils import timestamp_to_ms, ms_to_datetime  # Functions that convert datetime to ms since epoch and vice versa

Setup of Cognite SDK for communicating with CDF (will ask for API-key to Aker BPs 'prod'-environment in CDF, i.e. 'akerbp'):

In [2]:
api_key = getpass()
client = CogniteClient(
    api_key = api_key,
    project="akerbp",
    client_name="DSHub",
    base_url="https://api.cognitedata.com"
)

 ················································


### Class for fetching events from CDF

The following class is an example of how you could implement an "event fetcher" to be used in CrewLog. After the object is declared (`my_fetcher = CrewLogEventFetcher()`) and Aker BP assets are added (e.g. `my_fetcher.add_akerbp_asset('ULA', my_fetcher.get_abb_events, [<cdf-dataset-id>, ...], [<cdf-asset-id>, ...])`), you can fetch events from CDF like this: `events = my_fetcher.fetch_events('ULA')`. The first call will be made from the start of the current shift when the object was declared, e.g. 07:00 on the current day if the class was declared between 07:00 and 19:00. Succeeding `my_fetcher.fetch_events()` calls will only fetch events from the last time a call was made and to the current time of the call. 

E.g. If the object was declared at 14:00, the first call will fetch events from 07:00-14:00. If a second call is run at 16:00, it will only fetch events from 14:00-16:00.

In [3]:
# Class made to fetch events from CDF for CrewLog
class CrewLogEventFetcher:
    
    # Initialization function
    def __init__(self):
        self.switcher = {}
        self.data_set_ids = {}
        self.asset_subtree_ids = {}
        self.last_timestamps = {}
    
    
    # Adds an Aker BP asset to fetch events from
    # Parameters:
    # - 'akerbp_asset_abbr' must be a three letter Aker BP asset abbrieviation (e.g. 'ULA')
    # - 'get_events_function' must be a function like e.g. 'get_abb_events'
    # - 'data_set_ids' must be a list of CDF data set IDs (e.g. [123, 456, ...])
    # - 'asset_ids' must be a list of CDF asset IDs (e.g. [123, 456, ...])
    #
    def add_akerbp_asset(self, akerbp_asset_abbr, get_events_function, data_set_ids = None, asset_ids = None):
        
        # Declare essentials
        self.switcher[akerbp_asset_abbr] = get_events_function    # Asset specific function (e.g. self.get_abb_events)
        self.data_set_ids[akerbp_asset_abbr] = data_set_ids       # List of relevant CDF dataset ids
        self.asset_subtree_ids[akerbp_asset_abbr] = asset_ids     # List of relevant CDF asset ids
        
        # Set the last time an API call was made to be the start of the current shift
        # i.e. 0700 for day shift, 1900 for night shift
        timeref = datetime.datetime.today()
        if timeref.hour < 7:
            timeref = timeref - datetime.timedelta(days=1)
            self.last_timestamps[akerbp_asset_abbr] = datetime.datetime(timeref.year, timeref.month, timeref.day, 19)
        elif timeref.hour > 7 and timeref.hour < 19:
            self.last_timestamps[akerbp_asset_abbr] = datetime.datetime(timeref.year, timeref.month, timeref.day, 7)
        else:
            self.last_timestamps[akerbp_asset_abbr] = datetime.datetime(timeref.year, timeref.month, timeref.day, 19)
    
    
    # Function that returns the start_time for an event - for sorting purposes
    # - 'event' must be a CDF event object
    #
    def sort_by_start_time(self, event):
        return event.start_time
    
    
    # Shared function for fetching control system events from CDF - independent of control system supplier
    # The control system specific functions below makes use of this function
    # Parameters:
    # - 'akerbp_asset_abbr' must be a three letter Aker BP asset abbrieviation (e.g. 'ULA')
    # - 'metadata_sets' must be a list of dictionaries (e.g. [{}, ...] - see get_abb_events() function)
    # - 'from_time' is optional, but must be a datetime-object of the time you want to fetch events FROM
    # - 'to_time' is optional, but must be a datetime-object of the time you want to fetch event TO
    #
    def get_cdf_events(self, akerbp_asset_abbr, metadata_sets, from_time = None, to_time = None):        
        events = []
        
        # If function call has no 'from_time' or 'to_time', a standard time interval will be used in the API call
        # Standard time interval will be from the last time an API call was made to the current time, to eliminate duplicates
        if not from_time: from_time = self.last_timestamps[akerbp_asset_abbr]
        if not to_time: to_time = datetime.datetime.now()
        
        # Because of API limitations we need to run multiple API calls when we're looking for events that can hold 
        # different attributes or values. Each set of metadata in metadata_sets represents a single API call.
        for metadata in metadata_sets:
            
            # Loop through sets of asset subtree ids (max. 100 assets in a single API call)
            i = 0
            while i < len(self.asset_subtree_ids[akerbp_asset_abbr]):
                # End index to ensure a maximum of 100 assets for each call
                j = min(i+100, len(self.asset_subtree_ids[akerbp_asset_abbr]))
                
                # API call to CDF - fetching all events matching the given parameters
                temp_events = client.events.list(
                    start_time = {
                        'min': timestamp_to_ms(from_time),
                        'max': timestamp_to_ms(to_time)
                    },
                    metadata = metadata,
                    asset_subtree_ids = self.asset_subtree_ids[akerbp_asset_abbr][i:j],
                    data_set_ids = self.data_set_ids[akerbp_asset_abbr],
                    sort = ['startTime:asc'],
                    limit = -1
                )
                # Store the results from all individual API calls in one list of events
                events.extend(temp_events)
                
                # Update start index for asset list to be used in next call
                i += 100
        
        # Update the variable that keeps track of when the last time an API call was made
        # so that only events from this time is fetched for the next API call
        self.last_timestamps[akerbp_asset_abbr] = to_time
        
        # Sort events by start_time (chronological order)
        events.sort(key=self.sort_by_start_time)
        
        # Return sorted list of events
        return events
    
    
    # Individual function for ABB events, in case other control systems need unique filters
    # - 'akerbp_asset_abbr' must be a three letter Aker BP asset abbrieviation (e.g. 'ULA')
    #
    def get_abb_events(self, akerbp_asset_abbr):
        
        # Filters to be used in API call(s)
        abb_metadata_pre_filters = {
            'severity': ['801', '802', '803', '804', '809',
                         '901', '902', '903', '904', '909'],
            'newState': ['3']
        }
        # Valhall has two more systems and therefore more severity codes
        if akerbp_asset_abbr == 'VAL': 
            abb_metadata_pre_filters['severity'].extend(['805', '806', '905', '906'])
        
        # Filters to be used on API result
        abb_metadata_post_filters = {
            'alarmState': ['ACT'],
            'AlarmState': ['ACT']
        }
        
        # Build sets of metadata filters to be used in API call(s)
        # NOTE: add an empty dictionary to 'metadata_sets' if no metadata filters are needed
        metadata_sets = []
        for severity in abb_metadata_pre_filters['severity']:
            for new_state in abb_metadata_pre_filters['newState']:
                metadata = {
                    'severity': severity,
                    'newState': new_state
                }
                metadata_sets.append(metadata)
        
        # API call(s)
        res_events = self.get_cdf_events(akerbp_asset_abbr, metadata_sets)
        
        # Filter API results
        events = res_events.copy()
        for event in res_events:
            
            # Loop through the filters to be used on the API result
            for key, values in abb_metadata_post_filters.items():
                attribute = event.metadata.get(key)
                
                # If an event does not hold the desired value - remove said event from the results
                if attribute and attribute not in values:
                    events.remove(event)
                    break
        
        # Return filtered list of events
        return events
    
    
    # ---- TO BE IMPLEMENTED --------------------------------------------------------
    # NOTE: See 'get_abb_events()' function for an example on how to implement this
    # -------------------------------------------------------------------------------
    # Individual function for Kongsberg events, in case other control systems need unique filters
    def get_kongsberg_events(self, akerbp_asset_abbr):
        # Build sets of metadata filters to be used in API call(s)
        metadata_sets = []
        # API call(s)
        res_events = self.get_cdf_events(akerbp_asset_abbr, metadata_sets)
        # Filter API results
        events = res_events
        # Return list of filtered events
        return events
    
    
    # ---- TO BE IMPLEMENTED --------------------------------------------------------
    # NOTE: See 'get_abb_events()' function for an example on how to implement this
    # -------------------------------------------------------------------------------
    # Individual function for Siemens events, in case other control systems need unique filters
    def get_siemens_events(self, akerbp_asset_abbr):
        # Build sets of metadata filters to be used in API call(s)
        metadata_sets = []
        # API call(s)
        res_events = self.get_cdf_events(akerbp_asset_abbr, metadata_sets)
        # Filter API results
        events = res_events
        # Return list of filtered events
        return events
    
    
    # Print error message if an Aker BP asset have not been added to CrewLogEventFetcher
    def error_function(self, akerbp_asset_abbr):
        print(f"Asset '{akerbp_asset_abbr}' was not found.")
    
    
    # Main function to fetch all events from the given Aker BP asset since last time function was run
    # - 'akerbp_asset_abbr' must be a three letter Aker BP asset abbrieviation (e.g. 'ULA')
    #
    def fetch_events(self, akerbp_asset_abbr):
        return self.switcher.get(akerbp_asset_abbr, self.error_function)(akerbp_asset_abbr)
    

### Class for generating CDF asset list

The following classes are made to generate lists of asset IDs, to be used in e.g. `my_fetcher.add_akerbp_asset(..., asset_ids)`.

After a `CrewLogAssetFetcher` object is declared (`my_asset_fetcher = CrewLogAssetFetcher()`) and Aker BP assets are added (e.g. `my_asset_fetcher.add_akerbp_asset('ULA', '<.csv-filename>', [<cdf-dataset-id>, ...])`), you can fetch CDF a list of asset IDs from CDF like this: `asset_ids = my_asset_fetcher.fetch_assets('ULA')`. This list of asset IDs is to be used when adding Aker BP assets to the `CrewLogEventFetcher` object mentioned above.

In [4]:
# This is just a class to make code a bit more readable
class CrewLogTag():
    
    # Must have a tag name as parameter and optionally a list of alternative tag names (e.g. ABB control system tag for an Aker BP tag)
    def __init__(self, tag_name, alt_tag_names = []):
        self.name = tag_name
        self.alt_tags = alt_tag_names

In [9]:
# Class made to generate list of relevant CDF assets
class CrewLogAssetFetcher():
    
    # Initialization function
    def __init__(self):
        self.csv_filenames = {}
        self.data_set_ids = {}
        self.asset_ids = {}
    
    
    # Add an Aker BP asset to fetch events from
    # Parameters:
    # - 'akerbp_asset_abbr' must be a three letter Aker BP asset abbrieviation (e.g. 'ULA')
    # - 'csv_filename' must be a .csv-file in the 'data' directory (e.g. 'main_components_ula.csv')
    # - 'data_set_ids' must be a list of CDF data set IDs (e.g. [123, 456, ...])
    #
    def add_akerbp_asset(self, akerbp_asset_abbr, csv_filename, data_set_ids = None):
        
        # Declare essentials
        self.csv_filenames[akerbp_asset_abbr] = csv_filename      # List of relevant CDF asset ids
        self.data_set_ids[akerbp_asset_abbr] = data_set_ids       # List of relevant CDF dataset ids
    
    
    # This function reads all tags from the given .csv-file and creates a list of CrewLogTag objects
    # that has a name and possibly a list of alternative tag names (e.g. ABB tags)
    # - 'csv_filename' must be a .csv-file in the 'data' directory (e.g. 'main_components_ula.csv')
    #
    def import_tags_from_csv(self, csv_filename):
        with open(f'../data/input/{csv_filename}', 'rt', encoding='utf-8-sig') as f:
            data = csv.reader(f, delimiter=";")    # .csv-file must be semicolon-delimited
            header = next(data)                    # Remove header from dataset
            
            # Loop through all rows in .csv-file
            tag_names = []
            for row in data:
                alternative_tags = []
                if len(row) > 4:
                    alternative_tags = [str(column).strip() for column in row[4:] if column]
                tag_names.append(CrewLogTag(str(row[2]).strip(), alternative_tags))
                
            print(f"\nExtracted {len(tag_names)} main component(s) from '{csv_filename}'")
            return tag_names
    
    
    # This function fetches CDF assets matching with the given list of CrewLogTag objects. It should only return assets
    # that are relevant for the given data set IDs, meaning that it is either an asset, or has sub-assets that are part
    # of those data sets, or has events from those same data sets
    # Parameters:
    # - 'tag_names' must be a list of CrewLogTag objects (e.g. [CrewLogTag('NN-AA-NNNNN'), ...])
    #
    def get_cdf_asset_ids(self, tag_names, data_set_ids):
        print(f'Fetching CDF assets from data set(s) {data_set_ids}...')
        asset_ids = []
        
        # Loop through list of tag names and search CDF for matching assets
        for tag in tag_names:
            # List all CDF assets matching the given tag name
            assets = client.assets.list(name=tag.name, limit=-1)
            #print(f'Found {len(assets)} assets searching for {tag.name}: {[asset.id for asset in assets]}')
            tag_assets = [assets.copy()]
            
            # Check relevance with data set ids and removes assets that have no relevance to those data sets
            if data_set_ids:
                for asset in assets:
                    # If asset is not part of data sets
                    if asset.data_set_id not in data_set_ids:
                        # If asset does not have sub assets within those same data sets
                        if not client.assets.list(asset_subtree_ids=[asset.id], data_set_ids=data_set_ids, limit=1):
                            # If asset, or its sub-assets, does not have events within those same data sets - remove that asset
                            if not client.events.list(asset_subtree_ids=[asset.id], data_set_ids=data_set_ids, limit=1):
                                tag_assets[0].remove(asset)
            
            # If no assets were found relevant - use alternative tags if available
            if not tag_assets[0] and tag.alt_tags:
                #print(f'Tag has alternative tags: {tag.alt_tags}')
                for alt_tag in tag.alt_tags:
                    tag_assets.append(client.assets.list(name=alt_tag, data_set_ids=data_set_ids, limit=-1))
                #print(f'Remaining assets after considering alternative tags: {[asset.id for asset in tag_assets[0]]}')
            
            # If there's (still) duplicates - check if any of the duplicates have relations with eachother
            # and remove unneccessary duplicates
            temp_assets = tag_assets.copy()
            for i, asset_list in enumerate(temp_assets):
                if len(asset_list) > 1:
                    duplicate_ids = [asset.id for asset in asset_list]
                    for asset in asset_list:
                        temp_asset = asset
                        while temp_asset.parent_id:
                            if temp_asset.parent_id in duplicate_ids:
                                tag_assets[i].remove(temp_asset.parent())
                                duplicate_ids.remove(temp_asset.parent_id)
                                break
                            temp_asset = temp_asset.parent()
                    if len(duplicate_ids) > 1: print(f'Could not identify duplicates among {duplicate_ids}')
            
            # Return a list of asset IDs
            for asset_list in tag_assets:
                for asset in asset_list:
                    asset_ids.append(asset.id)
                    
        print(f'Found {len(asset_ids)} assets.')
        return asset_ids
    
    
    # This function exports asset IDs and names to respective .csv-files in 'output' directory
    def generate_csv_files(self):
        
        # Generate a .csv-file for each Aker BP asset
        print(f'\nGenerating .csv-files ...')
        for akerbp_asset_abbr, asset_ids in self.asset_ids.items():
            with open(f'output/asset_ids_{akerbp_asset_abbr.lower()}.csv', 'w') as f:
                writer = csv.writer(f)
                writer.writerow(['ASSET_IDS', 'ASSET_NAME'])   # Header of .csv-file
                for asset_id in asset_ids:
                    row = [asset_id, client.assets.retrieve(id=asset_id).name]
                    writer.writerow(row)
        print(f'.csv-files generated.')
    
    
    # Main function to fetch list of the found CDF asset IDs matching a given tag list, to be used with e.g. CrewLogEventFetcher()
    # - 'akerbp_asset_abbr' must be a three letter Aker BP asset abbrieviation (e.g. 'ULA')
    #
    def fetch_assets(self, akerbp_asset_abbr):
        tag_names = self.import_tags_from_csv(self.csv_filenames[akerbp_asset_abbr])
        self.asset_ids[akerbp_asset_abbr] = self.get_cdf_asset_ids(tag_names, self.data_set_ids[akerbp_asset_abbr])
        return self.asset_ids[akerbp_asset_abbr]
    

### Set up event fetcher

Full example for setting up event fetching from Ula:
```python
my_asset_fetcher = CrewLogAssetFetcher()
my_asset_fetcher.add_akerbp_asset('ULA', 'main_components_ula_abb_tags.csv', [2086908079872503])

asset_ids = my_asset_fetcher.fetch_assets('ULA')

my_fetcher = CrewLogEventFetcher()
my_fetcher.add_akerbp_asset('ULA', my_fetcher.get_abb_events, [2086908079872503], asset_ids)

events = my_fetcher.fetch_events('ULA')   # Fetches events in the time interval [shift start, current time]
```
The last line of code above will fetch all events from the start of the current shift up until the current time. To fetch new events you simply need to run the same line of code again at another time:
```python
events = my_fetcher.fetch_events('ULA')   # Fetches events in the time interval [last fetching time, current time]
```
NOTE: Remember that the `get_kongsberg_events` and `get_siemens_events` functions in the `CrewLogEventFetcher` class are yet to be implemented as their data is not available in CDF at this time.

In [10]:
# Create the CrewLogEventFetcher object to be used for fetching events from CDF
# NOTE: See the CrewLogEventFetcher class above for details on its functions
#
crewlog_event_fetcher = CrewLogEventFetcher()


# These are the main setup parameters needed for fetching events from an Aker BP asset from CDF
# All assets should have a 'get_function' from the CrewLogEventFetcher object, a list of 'cdf_data_set_ids' and
# a .csv-file with tag names for the given Aker BP asset
#
# NOTE: When e.g. Digital Foundation for Skarv becomes available in CDF, we must add its data set ID under
#      'cdf_data_set_ids' for 'SKA' below, just like we have done for 'ULA' and 'VAL'.
#
# CAUTION: The 'get_kongsberg_events' and 'get_siemens_events' functions are yet to be implemented in the
#          CrewLogEventFetcher class. Use the 'get_abb_events' as an example when implementing these when
#          Skarv, Alvheim and Ivar Aasen data becomes avaible in CDF and you have identified how their data
#          must be filtered.
#
akerbp_assets = {
    'ULA': {
        'get_function': crewlog_event_fetcher.get_abb_events,        # Ula has ABB control system
        'cdf_data_set_ids': [2086908079872503],                      # OPC UA data ULA (Digital Foundation dataset)
        'csv_filename': 'main_components_ula_abb_tags.csv'
    },
    'VAL': {
        'get_function': crewlog_event_fetcher.get_abb_events,        # Valhall has ABB control system
        'cdf_data_set_ids': [140572846698809],                       # OPC UA data VAL (Digital Foundation dataset)
        'csv_filename': 'main_components_valhall.csv'
    },
    'SKA': {
        'get_function': crewlog_event_fetcher.get_kongsberg_events,  # Skarv has Kongsberg control system
        'cdf_data_set_ids': [],
        'csv_filename': 'main_components_skarv.csv'
    },
    'ALV': {
        'get_function': crewlog_event_fetcher.get_kongsberg_events,  # Alvheim has Kongsberg control system
        'cdf_data_set_ids': [],
        'csv_filename': 'main_components_alvheim.csv'
    },
    'IAA': {
        'get_function': crewlog_event_fetcher.get_siemens_events,    # Ivar Aasen has Siemens control system
        'cdf_data_set_ids': [],
        'csv_filename': 'main_components_ivar_aasen.csv'
    }
}

# Create the CrewLogAssetFetcher object to be used for generating lists of relevant CDF asset IDs
# NOTE: See the CrewLogAssetFetcher class above for details on its functions
#
crewlog_asset_fetcher = CrewLogAssetFetcher()

for akerbp_asset_abbr, asset_data in akerbp_assets.items():
    # Adds an Aker BP asset to the CrewLogAssetFetcher object
    crewlog_asset_fetcher.add_akerbp_asset(
        akerbp_asset_abbr,
        asset_data['csv_filename'],
        asset_data['cdf_data_set_ids']
    )
    
    # Fetches a list of asset IDs from the given asset
    asset_data['cdf_asset_ids'] = crewlog_asset_fetcher.fetch_assets(akerbp_asset_abbr)
    
    # Adds and Aker BP asset to the CrewLogEventFetcher object with the parameters declared in 'akerbp_assets' above
    # as well as the list of asset IDs from the previous line
    crewlog_event_fetcher.add_akerbp_asset(
        akerbp_asset_abbr,
        asset_data['get_function'],
        asset_data['cdf_data_set_ids'],
        asset_data['cdf_asset_ids']
    )
    
# Generates a .csv-file for each Aker BP asset with a list of their respective CDF asset ids
# NOTE: These are probably files that the backend in CrewLog needs when they will be implementing their own functionality
crewlog_asset_fetcher.generate_csv_files()



Extracted 219 main component(s) from 'main_components_ula_abb_tags.csv'
Fetching CDF assets from data set(s) [2086908079872503]...
Found 175 assets.

Extracted 462 main component(s) from 'main_components_valhall.csv'
Fetching CDF assets from data set(s) [140572846698809]...
Found 366 assets.

Extracted 517 main component(s) from 'main_components_skarv.csv'
Fetching CDF assets from data set(s) []...
Found 501 assets.

Extracted 233 main component(s) from 'main_components_alvheim.csv'
Fetching CDF assets from data set(s) []...
Could not identify duplicates among [4909397956766959, 5576109327626046]
Could not identify duplicates among [1361192871416397, 2175262956095244]
Could not identify duplicates among [4887147706554168, 6680966522712542]
Could not identify duplicates among [5794600446063807, 7313151598201456]
Could not identify duplicates among [1515210526033923, 3969124812466527]
Found 237 assets.

Extracted 71 main component(s) from 'main_components_ivar_aasen.csv'
Fetching CDF as

### Other functions

In [11]:
# This function retrieves a CDF event by its event id
def get_event(id):
    return client.events.retrieve(id=id)

# This function retrieves a CDF asset by its asset id
def get_asset(id):
    return client.assets.retrieve(id=id)

In [12]:
# This function extracts tag names from a text string
# Inputs:
# string - text string (e.g. '23-KA-9103-M01_CB_travel_alarm')
# field - all caps abbreviation of an Aker BP asset (e.g. 'VAL')
#
# Output: 
# list of found tag names in string, matching the given fields tagging convention
# (e.g. ['23-KA-9103'])

def extract_tag_names(string, field):
    def default(string, field):
        print(f"There is no function for extracting tags for {field}")
        return None
    
    # Ula tags usually follow this format: A(AAA)-NNNN(N)(A)-(A)
    def extract_ula_tag(string, field):
        return re.search(r'[A-Z]+-\d+([A-Z][A-Z]?|(-[A-Z]\b)?)', string)
    
    # Valhall tags usually follow this format: NN-AA(A)-NNNN(NN)(A)
    def extract_valhall_tag(string, field):
        return re.search('[0-9][0-9]-[A-Z]+-\d+[A-Z]?[A-Z]?', string)
    
    switcher = {
        'ULA': extract_ula_tag,
        'VAL': extract_valhall_tag
    }
    
    x = re.split('_', string)
    matches = []
    for word in x:
        y = switcher.get(field, default)(word, field)
        if y: 
            matches.append(y.group())
    if not matches: print(f'Could not identify any tags with {field} tagging convention in "{string}"')
    
    return matches

In [13]:
# This function looks for the field (Aker BP asset) of a CDF asset
def get_field_from_asset(asset):
    # The root asset usually holds information about the Aker BP installation
    root_asset = get_asset(asset.root_id)
    installation = root_asset.name
    
    if installation in ['VAL', 'VFN', 'VFS', 'VFW', 'HOD', 'HOP', 'VLA']:
        return 'VAL'
    elif installation in ['ULA', 'TAM']:
        return 'ULA'
    elif installation in ['SKA', 'ALV', 'IAA']:
        return installation
    
    print(f'Found unexpected field from root asset: {installation}')
    return installation

In [14]:
# This function recursively searches for a parent asset of an event that is not an asset created by e.g. signal tags/subtags/softtags/etc.
# 
# Example hierarchy:
# Z-0501 (id: 8289785573229588)          <-- True parent
#  LP-0561 (id: 4103300555352816)        <-- Some other ancestor (tag does not match)
#    Z-0501-01 (id: 6791812333680596)    <-- Grandparent that comes from Aveva (still has signal tag)
#      Z-0501-01 (id: 4728001758341600)  <-- Parent asset in OPC UA dataset (has signal tag)
#        (event) Abnormal condition (source: Z-0501-01, id: 7491014714576560)

def get_true_parent(event, print_hierarchy = False):
    parent = None
    source_name = event.metadata["source"]
    
    # Store order of hierarchy for printing and troubleshooting purposes
    hierarchy = [[f'(event) {event.description} (source: {source_name}, id: {event.id})'] for _ in range(len(event.asset_ids))]
    
    # Loop through all related CDF assets for the given event
    for i, asset_id in enumerate(event.asset_ids):
        temp_asset = get_asset(asset_id)
        
        # List of potential tag names following the given fields tagging convention, usually just one
        tag_names = extract_tag_names(source_name, get_field_from_asset(temp_asset))   
        
        # Recursively search for a parent asset that matches with the desired tag name (up the CDF asset hierarchy)
        j = 0
        while temp_asset.parent_id and not parent and j < 5:
            hierarchy[i].append(f'{temp_asset.name} (id: {temp_asset.id})')    # For printing and troubleshooting purposes
            for name in tag_names:
                # If the parent assets name is part of or equal to the desired tag name, we have found a match
                # (e.g. if the found asset has a main equipment tag name, but the desired tag name is a redundancy tag with A/B/etc. at the end, we still consider it a match)
                if temp_asset.name in name:
                    parent = temp_asset
                    if name != temp_asset.name: print(f'{temp_asset.name} will be used instead of {name}')   # To verify that non-exact matches still are relatives
                    
                    # For printing and troubleshooting purposes
                    if print_hierarchy:
                        for k in range(len(hierarchy[i])):
                            indent = k*"  "
                            print(indent + hierarchy[i][-(k+1)])
                            
                    return parent
            temp_asset = get_asset(temp_asset.parent_id)
            j += 1
        
        #if not parent: print(f'Could not find true parent of event {event.id} after {j} recursive steps using asset {asset_id} ({i+1}/{len(event.asset_ids)})')
    #print(f'No true parents were found for event {event.id} (event had {len(event.asset_ids)} related CDF asset(s))')
    
    #if print_hierarchy: ...
    
    return parent

In [15]:
# This function recursively searches for a parent asset that holds information that can determine a SAP FLOC prefix
def get_floc_prefix(asset):
    temp_asset = asset
    
    # FLOC prefix can often be found from an assets 'PLATFORM CODE' attribute
    prefix = temp_asset.metadata.get('PLATFORM CODE', '')
    
    # Recursively search for a parent asset holding the 'PLATFORM CODE' property (up the CDF asset hierarchy)
    i = 0
    while temp_asset.parent_id and not prefix and i < 5:
        temp_asset = get_asset(temp_asset.parent_id)
        prefix = temp_asset.metadata.get('PLATFORM CODE', '')
        i += 1
    
    #if not prefix: print(f'Could not find FLOC prefix for asset {asset.id} after {i} recursive steps')
    return prefix

In [16]:
# This function returns the SAP FLOC of a CDF event
def get_floc(event, print_hierarchy = False):
    temp_floc = event.metadata["source"]
    true_parent = get_true_parent(event, print_hierarchy)
    if true_parent:
        temp_floc = true_parent.name
    prefix = get_floc_prefix(true_parent if true_parent else get_asset(event.asset_ids[0]))
    if prefix:
        temp_floc = prefix + "-" + temp_floc
    return temp_floc

In [17]:
# This function prints an event with the following format:
#  <time> | <floc> | <object-description> | <event-description> | <severity> (<event-id>)
def print_event(event):
    time = event.metadata.get('activeTime', event.metadata['time'])
    floc = get_floc(event)
    object_desc = event.metadata.get('ObjectDescription', '**no object description**')
    description = f'{event.metadata["message"]}, {event.metadata.get("conditionName", "")}'
    severity = event.metadata['severity']
    print(f'    {time} | {floc} | {object_desc} | {description} | {severity} (id: {event.id})')

### Test of fetching alarms

In [19]:
akerbp_asset_abbr = 'VAL'
timeref = datetime.datetime.now()
#crewlog_event_fetcher.last_timestamps[akerbp_asset_abbr] -= datetime.timedelta(hours=12)
events = crewlog_event_fetcher.fetch_events(akerbp_asset_abbr)
delay = datetime.datetime.now() - timeref

print(f'Time elapsed while fetching events from {akerbp_asset_abbr}: {delay}')
if events:
    for event in events:
        print_event(event)
else:
    print('No events found.')

Time elapsed while fetching events from VAL: 0:01:19.865211
    9/29/2022 7:07:29 AM | VIP-48-NXI-82060G | IP XD8001 HighFuel | Alarm HH, High High | 902 (id: 7965098300933456)
    9/29/2022 7:07:31 AM | VIP-48-NXI-82060C | IP XD8001 Alarm | Alarm HH, High High | 902 (id: 1424355083939100)
    9/29/2022 7:08:42 AM | VIP-48-NXI-82060F | IP XD8001 LowFuel | Alarm HH, High High | 902 (id: 6246370064110145)
    9/29/2022 7:39:19 AM | VPH-11-TIC-92306 | PH Interstg Htr Oil Out | <= 55.0 ºC Warning L, Low | 804 (id: 4185713949259738)
    9/29/2022 7:53:01 AM | VPH-32-LT-95807 | PH TEG Surge Drum | <= 27.0 % Warning L, Low | 804 (id: 8324375927166473)
    9/29/2022 7:59:40 AM | VPH-43-PT-96675 | PH TEG Transfer Pump Disch | <= 2.0 bar Warning L, Low | 804 (id: 5212396431861260)
